In [1]:
import sys, os
import torch, wandb
import torch.nn as nn
from torch.utils.data import DataLoader
sys.path.append(os.path.abspath(os.path.join(os.curdir, '..')))

In [2]:
from configs import unet_convnextv2_config as config
from models.unet_convnextv2 import Unet
from datasets.depth_dataset import DepthDataset
from utils.train_utils import train_model

/home/v.lomtev/audio_ml/ml_env_3.10_maxvit/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
# Set a fixed random seed for reproducibility
torch.manual_seed(config.random_seed)

train_full_dataset = DepthDataset(
    data_dir=os.path.join(config.dataset_path, 'train/train'),
    list_file=os.path.join(config.dataset_path, 'train_list.txt'), 
    transform=config.transform_train,
    target_transform=config.target_transform,
    has_gt=True)
    
    # Create test dataset without ground truth
test_dataset = DepthDataset(
    data_dir=os.path.join(config.dataset_path, 'test/test'),
    list_file=os.path.join(config.dataset_path, 'test_list.txt'),
    transform=config.transform_val,
    has_gt=False)  # Test set has no ground truth
    
# Split training dataset into train and validation
total_size = len(train_full_dataset)
train_size = int((1-config.val_part) * total_size)  
val_size = total_size - train_size    
    
train_dataset, val_dataset = torch.utils.data.random_split(
    train_full_dataset, [train_size, val_size]
)
#val_dataset.transform = config.transform_val # I dont think we need to use augmentations for validation

# Create data loaders with memory optimizations
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.train_bs, 
    shuffle=True, 
    num_workers=config.num_workers, 
    pin_memory=True,
    drop_last=True,
    persistent_workers=True
)
    
    
val_loader = DataLoader(
    val_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)
    
test_loader = DataLoader(
    test_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)

print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")

Train size: 20375, Validation size: 3596, Test size: 650


In [4]:
model = config.model()
# #model = nn.DataParallel(model)


optimizer = config.optimizer(model.parameters())
print(f"Using device: {config.device}")

Using device: cuda:3


In [ ]:
print("Starting training...")
exp_name = "convnextv2"
with wandb.init(project="CIL",
                save_code=True,
                notes=config.WANDB_NOTES):
    model = train_model(model, train_loader, val_loader,
                        config.loss, optimizer, config.epochs, config.device,
                       exp_path=os.path.join(config.dataset_path, exp_name))

Starting training...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: noloo. Use `wandb login --relogin` to force relogin


Epoch 1/30


Evaluating: 100%|███████████████████████████████████████████████████████████████| 225/225 [00:47<00:00,  4.74it/s]


Train Loss: 1.8804, Validation Loss: 1.4468
New best model saved at epoch 1 with validation loss: 1.4468
Epoch 2/30


...

Evaluating: 100%|███████████████████████████████████████████████████████████████| 225/225 [00:47<00:00,  4.72it/s]


Train Loss: 0.4751, Validation Loss: 0.3975
New best model saved at epoch 5 with validation loss: 0.3975
Epoch 6/30


Evaluating: 100%|███████████████████████████████████████████████████████████████| 225/225 [00:47<00:00,  4.75it/s]


Train Loss: 0.3665, Validation Loss: 0.3094
New best model saved at epoch 6 with validation loss: 0.3094
Epoch 7/30


Training:  30%|██████████████████▉                                             | 377/1273 [03:02<07:12,  2.07it/s]

In [ ]:
import utils.train_utils as tu

In [ ]:
from utils.train_utils import evaluate_model
import importlib
importlib.reload(tu)
tu.evaluate_model(model, val_loader, config.device,
                  exp_path=os.path.join(config.dataset_path, exp_name))